# Calculate basic descriptors
Most data evaluation start with a pandas dataframe. So the rest of the group does not have to deal with the sql database i have decided to create this template so everyone can start form the pandas dataframe


In [4]:
import sqlite3
import pandas as pd
from Bio.SeqUtils import ProtParam
from rdkit import Chem
from rdkit.Chem import Descriptors
con = sqlite3.connect("../02-Data_Curation/unified.db")
cur = con.cursor()

df = pd.read_sql_query("SELECT * FROM prod", con)

# Calculate seq_length

In [6]:
df['seq_length'] = df['seq'].apply(lambda x: len(x))

# Calculate rdkit MOL object


In [7]:
df['mol'] = df['seq'].apply(lambda x: Chem.MolFromSequence(x))

,id,name,AB,description,OX,source,seq,valid,seq_length,mol
0,DRAMP02488; P0DI85,dramp_DRAMP02488; OXLA8_DEIAC,1,L-amino-acid oxidase ACTX-8 (Fragment),alien; Deinagkistrodon acutus OX=36307,dramp_antimicrobial.fasta; uniprot_swissprot.f...,ADDRNPLEEFRENNYEEFL,yes - merged duplicate,19,<rdkit.Chem.rdchem.Mol object at 0x7f9480921620>
1,ADAM_0078; P86520,InverPep_ADAM_0078; SBTXA_SOYBN,1,Soybean toxin 27 kDa chain (Fragment),alien; Glycine max OX=3847,InverPep.fasta; uniprot_swissprot.fasta,ADPTFGFTPLGLSEKANLQIMKAYD,yes - merged duplicate,25,<rdkit.Chem.rdchem.Mol object at 0x7f9480921690>
2,ADAM_0131; P81493,InverPep_ADAM_0131; DFAX1_BETVU,1,Defensin-like protein AX1,alien; Beta vulgaris OX=161934,InverPep.fasta; uniprot_swissprot.fasta,AICKKPSKFFKGACGRDADCEKACDQENWPGGVCVPFLRCECQRSC,yes - merged duplicate,46,<rdkit.Chem.rdchem.Mol object at 0x7f9480921700>
3,ADAM_0141; P84644,InverPep_ADAM_0141; CIRF_CHAPA,1,Circulin-F,alien; Chassalia parviflora OX=58431,InverPep.fasta; uniprot_swissprot.fasta,AIPCGESCVWIPCISAAIGCSCKNKVCYR,yes - merged duplicate,29,<rdkit.Chem.rdchem.Mol object at 0x7f94809214d0>
4,ADAM_0221; P80278,InverPep_ADAM_0221; DRS2_PHYSA,1,Dermaseptin-S2,alien; Phyllomedusa sauvagei OX=8395,InverPep.fasta; uniprot_swissprot.fasta,ALWFTMLKKLGTMALHAGKAALGAAANTISQGTQ,yes - merged duplicate,34,<rdkit.Chem.rdchem.Mol object at 0x7f9480921540>
...,...,...,...,...,...,...,...,...,...,...
11558,Q3E778,YB96B_YEAST,0,Uncharacterized protein YBR196C-B,Saccharomyces cerevisiae (strain ATCC 204508 /...,uniprot_swissprot.fasta,MWVVLSKEKILLKKAYYAKTILFSALVLRGVRGE,yes,34,<rdkit.Chem.rdchem.Mol object at 0x7f94803f7370>
11559,Q3E781,YB221_YEAST,0,Uncharacterized protein YBR221W-A,Saccharomyces cerevisiae (strain ATCC 204508 /...,uniprot_swissprot.fasta,MFSHFEVSENRPRKQPRRKRISLGMINTVVSLDR,yes,34,<rdkit.Chem.rdchem.Mol object at 0x7f94803f73e0>
11560,Q3E7Z3,YI018_YEAST,0,Uncharacterized protein YIR018C-A,Saccharomyces cerevisiae (strain ATCC 204508 /...,uniprot_swissprot.fasta,MPSDYTSHYPVILIKKKKKKIAGMYRHSKRYLEIMSTASAQFVGN,yes,45,<rdkit.Chem.rdchem.Mol object at 0x7f94803f7450>
11561,Q3E7Z6,YH015_YEAST,0,Uncharacterized protein YHL015W-A,Saccharomyces cerevisiae (strain ATCC 204508 /...,uniprot_swissprot.fasta,MTAFASLREPLVLANLKIKVHIYRMKR,yes,27,<rdkit.Chem.rdchem.Mol object at 0x7f94803f74c0>


# Calculate Descriptors

In [8]:
# Log P
df['logP'] =  df['mol'].apply(lambda x: Descriptors.MolLogP(x))

In [11]:
df['molecular weight'] = df['mol'].apply(lambda x: Descriptors.ExactMolWt(x))

In [17]:
def calculate_descriptors(mol):
    """Calculate all descriptors for a molecule."""
    descriptors = {desc_name: desc_func(mol) for desc_name, desc_func in Descriptors.descList}
    return descriptors

# Applying the function and creating a new DataFrame from the results
df_descriptors = df['mol'].apply(calculate_descriptors).apply(pd.Series)

# Concatenate the original DataFrame with the new descriptors DataFrame
df_final = pd.concat([df, df_descriptors], axis=1)

df_final.head()

KeyboardInterrupt: 

peptide descriptors

In [ ]:
import pandas as pd
from propy import PyPro
from Bio.SeqUtils.ProtParam import ProteinAnalysis
import time

# Record the start time
start_time = time.time()

# Your function to extract all features with detailed logging
def extract_all_features(peptide):
    pro = PyPro.GetProDes(peptide)
    features = []
    try:
        print(f"Processing sequence: {peptide}")
        
        # Amino acid composition
        aa_comp = pro.GetAAComp().values()
        print(f"AAComp for {peptide}: {aa_comp}")
        features += list(aa_comp)
        
        # Dipeptide composition
        dp_comp = pro.GetDPComp().values()
        print(f"DPComp for {peptide}: {dp_comp}")
        features += list(dp_comp)
        
        # Tripeptide composition
        tp_comp = pro.GetTPComp().values()
        print(f"TPComp for {peptide}: {tp_comp}")
        features += list(tp_comp)
        
        # Moreau-Broto autocorrelation descriptors
        moreau_broto_auto = pro.GetMoreauBrotoAuto().values()
        print(f"MoreauBrotoAuto for {peptide}: {moreau_broto_auto}")
        features += list(moreau_broto_auto)
        
        # Moran autocorrelation descriptors
        moran_auto = pro.GetMoranAuto().values()
        print(f"MoranAuto for {peptide}: {moran_auto}")
        features += list(moran_auto)
        
        # Geary autocorrelation descriptors
        geary_auto = pro.GetGearyAuto().values()
        print(f"GearyAuto for {peptide}: {geary_auto}")
        features += list(geary_auto)
        
        # Quasi-sequence order descriptors
        qso = pro.GetQSO().values()
        print(f"QSO for {peptide}: {qso}")
        features += list(qso)
        
        # Additional physicochemical properties
        analysed_seq = ProteinAnalysis(peptide)
        molecular_weight = analysed_seq.molecular_weight()
        print(f"Molecular weight for {peptide}: {molecular_weight}")
        isoelectric_point = analysed_seq.isoelectric_point()
        print(f"Isoelectric point for {peptide}: {isoelectric_point}")
        instability_index = analysed_seq.instability_index()
        print(f"Instability index for {peptide}: {instability_index}")
        gravy = analysed_seq.gravy()
        print(f"GRAVY for {peptide}: {gravy}")
        
        features += [
            molecular_weight,
            isoelectric_point,
            instability_index,
            gravy,
        ]
    except ZeroDivisionError:
        print(f"Division by zero error for sequence: {peptide}")
        return None
    except Exception as e:
        print(f"Error processing sequence {peptide}: {e}")
        return None
    return features

# Apply function to each peptide sequence
df['Features'] = df['seq'].apply(extract_all_features)

# Filter out sequences where feature extraction failed
df_filtered = df.dropna()

# Expand features into separate columns
df_features = pd.DataFrame(df_filtered['Features'].tolist())

# Concatenate original dataframe with features
df_final = pd.concat([df_filtered.drop(columns=['Features']), df_features], axis=1)

# Calculate the total time taken
total_time = time.time() - start_time
print(f"Total time taken: {total_time} seconds")

